In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def scrape_pages(topic):
  link = 'https://wuzzuf.net/search/jobs/?a=hpb%7Cspbg&q=' + topic.replace(' ', '%20')
  response = requests.get(link)
  soup = BeautifulSoup(response.content, 'lxml')

  while not soup.find('strong'):
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'lxml')

  num_of_jobs = int(soup.find('strong').text)
  num_of_pages = num_of_jobs // 15 + num_of_jobs % 15

  titles_lst, links_lst, occupations_lst, companies_lst, locations_lst, specs_lst = [], [], [], [], [], []

  for pageNum in range(num_of_pages):
    page_link = link + '&start=' + str(pageNum)
    page = requests.get(page_link)
    soup = BeautifulSoup(page.content, 'lxml')

    titles = soup.find_all("h2", 'css-m604qf')
    titles_lst += [title.a.text for title in titles]
    links_lst += [title.a['href'] for title in titles]

    occupations_types = ['On-site', 'Hybrid', 'Remote', 'Full Time', 'Internship', 'Part Time', 'Shift based', 'Freelance / Project']
    occupations = soup.find_all('div', 'css-1lh32fc')
    occupations_lst += [' | '.join([t for t in occupations_types if t in oc.text]) for oc in occupations]

    companies = soup.find_all('div', 'css-d7j1kk')
    companies_lst += [name.a.text.rstrip(' -') for name in companies]
    locations_lst += [location.span.text.strip() for location in companies]

    specs = soup.find_all("div", {'class': 'css-y4udm8'})
    specs_lst += [''.join(div.text.strip() for div in spec.find_all('div')[1:]) for spec in specs]

  scraped_data = {}
  scraped_data['Title'] = titles_lst
  scraped_data['Link'] = links_lst
  scraped_data['Occupation'] = occupations_lst
  scraped_data['Company'] = companies_lst
  scraped_data['Location'] = locations_lst
  scraped_data['Specs'] = specs_lst

  df = pd.DataFrame(scraped_data)
  return scraped_data, df

In [ ]:
def combine_dfs(dfs):
  df = pd.concat(dfs)
  df = df.drop_duplicates()
  return df

def combine_dicts(dicts):
  combined_dict = {}
  for key in dicts[0].keys():
    combined_dict[key] = []
    for dict in dicts:
      combined_dict[key] += dict[key]
  return combined_dict

In [ ]:
ds_dict, ds_df = scrape_pages('Data science')
ml_dict, ml_df = scrape_pages('Machine learning')
ai_dict, ai_df = scrape_pages('Artificial intelligence')

combined_dict = combine_dicts([ds_dict, ml_dict, ai_dict])
combined_df = combine_dfs([ds_df, ml_df, ai_df])

combined_df.to_csv('data.csv', index=False)